In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import resnet_model
import os

%matplotlib inline

/home/barney_awesome_li/anaconda3/envs/tfgpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [3]:
# Load training data set
train = pd.read_json('../data/train.json')

In [4]:
print(train.head())

                                              band_1  \
0  [-27.878360999999998, -27.15416, -28.668615, -...   
1  [-12.242375, -14.920304999999999, -14.920363, ...   
2  [-24.603676, -24.603714, -24.871029, -23.15277...   
3  [-22.454607, -23.082819, -23.998013, -23.99805...   
4  [-26.006956, -23.164886, -23.164886, -26.89116...   

                                              band_2        id inc_angle  \
0  [-27.154118, -29.537888, -31.0306, -32.190483,...  dfd5f913   43.9239   
1  [-31.506321, -27.984554, -26.645678, -23.76760...  e25388fd   38.1562   
2  [-24.870956, -24.092632, -20.653963, -19.41104...  58b2aaa0   45.2859   
3  [-27.889421, -27.519794, -27.165262, -29.10350...  4cfc3a18   43.8306   
4  [-27.206915, -30.259186, -30.259186, -23.16495...  271f93f4   35.6256   

   is_iceberg  
0           0  
1           0  
2           1  
3           0  
4           0  


In [5]:
# re-order the training set
# order_num = np.random.rand(train.shape[0])
# train['order'] = order_num
# train.sort_values('order', axis=0, inplace=True)
# print(train.head())

In [6]:
raw_y = train['is_iceberg'].tolist()
tensor_y = tf.one_hot(raw_y, depth=2)
with tf.Session() as sess:
    total_y = np.array(sess.run(tensor_y))
print(total_y.shape)

(1604, 2)


In [7]:
raw_x_1 = np.array(train['band_1'].tolist())
raw_x_2 = np.array(train['band_2'].tolist())
total_x = np.concatenate((raw_x_1, raw_x_2), axis = 1)
total_x = raw_x_1
print(total_x.shape)

(1604, 5625)


In [8]:
dev_x = total_x[0:300,:]
dev_y = total_y[0:300,:]
print(dev_x.shape)
train_x = total_x[300:,:]
train_y = total_y[300:,:]
print(train_x.shape)
print(train_y.shape)

(300, 5625)
(1304, 5625)
(1304, 2)


In [9]:
class MiniBatch(object):
    def __init__(self, x, y, batch_size):
        self.generator = self.slice_random(x, y, batch_size)
    
    def next_batch(self):
        try:
            return(next(self.generator))
        except StopIteration as e:
            return None, None
        
    @staticmethod    
    def slice_random(X, Y, mini_batch_size):
        tmp_X = X.copy()
        tmp_Y = Y.copy()
        cur_index = 0
        while(tmp_X.shape[0]>0):
            pick_size = tmp_X.shape[0] if tmp_X.shape[0]<mini_batch_size else mini_batch_size
            mini_batch_x = tmp_X[cur_index:cur_index + pick_size,:]
            mini_batch_y = tmp_Y[cur_index:cur_index + pick_size,:]
            cur_index += pick_size
            yield mini_batch_x, mini_batch_y

In [10]:
# test
test_mini_batch = MiniBatch(dev_x, dev_y, 50)


In [11]:
with tf.name_scope('reshape'):
    x_in = tf.placeholder(tf.float32)
    y_in = tf.placeholder(tf.float32)
    x = tf.image.resize_image_with_crop_or_pad(tf.reshape(x_in, [-1,75,75,1]), 256, 256) 
    y = tf.reshape(y_in, [-1, 2])

In [12]:
with tf.name_scope('resnet'):
    y_generator = resnet_model.imagenet_resnet_v2(18, 2, 'channels_last')
    y_ = y_generator(x, True)

In [13]:
with tf.name_scope('cost'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_, labels=y))

In [14]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer().minimize(cost)

In [15]:
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
    accuracy = tf.reduce_mean(correct_prediction)

In [16]:
# # normal training
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     tf.add_to_collection('accuracy', accuracy)
#     tf.add_to_collection('y_', y_)
#     tf.add_to_collection('x_in', x_in)
#     tf.add_to_collection('y_in', y_in)
#     # writer = tf.summary.FileWriter('./logs',sess.graph)
#     for i in range(1001):
#         minibatch = MiniBatch(train_x, train_y, 64)
#         x_batch,y_batch = minibatch.next_batch()
#         if x_batch is None or x_batch.shape[0]<64:
#             minibatch = MiniBatch(train_x, train_y, 64)
#             x_batch,y_batch = minibatch.next_batch()
#         # print('minibatch size {} and {}'.format(x_batch.shape, y_batch.shape))
#         if i%100 == 0:
#             testMiniBatch = MiniBatch(dev_x, dev_y, 64)
#             x_batch_test, y_batch_test = testMiniBatch.next_batch()
#             if x_batch_test is None or x_batch_test.shape[0]<64:
#                 testMiniBatch = MiniBatch(dev_x, dev_y, 64)
#                 x_batch_test, y_batch_test = testMiniBatch.next_batch()
#             train_accuracy = accuracy.eval(session=sess,
#                                             feed_dict={
#                                                 x_in:x_batch_test, y_in:y_batch_test
#                                             })
#             print('step {}, training accuracy {}'.format(i, train_accuracy))
#             print('test accuracy {}'.format(accuracy.eval(session=sess, feed_dict={x_in: dev_x , y_in: dev_y})))
#             saver = tf.train.Saver()
#             saved_path = saver.save(sess, './models/{}.ckpt'.format(i))
#             print('model saved to {}'.format(saved_path))
#         sess.run(optimizer, feed_dict={x_in:x_batch, y_in:y_batch})
#     # writer.close()
#     print('test accuracy {}'.format(accuracy.eval(session=sess, feed_dict={x_in: dev_x , y_in: dev_y})))
    

In [ ]:
# # training with 2 image
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     tf.add_to_collection('accuracy', accuracy)
#     tf.add_to_collection('y_', y_)
#     tf.add_to_collection('x_in', x_in)
#     tf.add_to_collection('y_in', y_in)
#     minibatch = MiniBatch(train_x, train_y, 64)
#     x_batch,y_batch = minibatch.next_batch()
#     for i in range(1001):
#         if i%10 == 0:
#             train_accuracy = accuracy.eval(session=sess,
#                                             feed_dict={
#                                                 x_in:x_batch, y_in:y_batch
#                                             })
#             print('step {}, training accuracy {}'.format(i, train_accuracy))
#             saver = tf.train.Saver()
#             saved_path = saver.save(sess, './models/{}.ckpt'.format(i))
#             print('model saved to {}'.format(saved_path))
#         sess.run(optimizer, feed_dict={x_in:x_batch, y_in:y_batch})
#     print('test accuracy {}'.format(accuracy.eval(session=sess, feed_dict={x_in: dev_x , y_in: dev_y})))

In [ ]:
# batch gradient descent without random shuffle
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    tf.add_to_collection('accuracy', accuracy)
    tf.add_to_collection('y_', y_)
    tf.add_to_collection('x_in', x_in)
    tf.add_to_collection('y_in', y_in)
    # epoch
    batch_size = 64
    for i in range(1001):
        for j in range(int(train_x.shape[0] / 64)):
            x_batch = train_x[j * batch_size: (j+1) * batch_size]
            y_batch = train_y[j * batch_size: (j+1) * batch_size]
            sess.run(optimizer, feed_dict={x_in:x_batch, y_in:y_batch})
        # print accuracy after each epoch
        train_accuracy = accuracy.eval(session=sess,
                                        feed_dict={
                                            x_in:x_batch, y_in:y_batch
                                        })
        print('epoch {}, training accuracy {}'.format(i, train_accuracy))
        print('test accuracy {}'.format(accuracy.eval(session=sess, feed_dict={x_in: dev_x , y_in: dev_y})))
        saver = tf.train.Saver()
        saved_path = saver.save(sess, './models/{}.ckpt'.format(i))
        print('model saved to {}'.format(saved_path))

epoch 0, training accuracy 1.0
test accuracy 0.5
model saved to ./models/0.ckpt
epoch 1, training accuracy 0.234375
test accuracy 0.7166666388511658
model saved to ./models/1.ckpt
epoch 2, training accuracy 0.796875
test accuracy 0.653333306312561
model saved to ./models/2.ckpt
epoch 3, training accuracy 0.828125
test accuracy 0.7366666793823242
model saved to ./models/3.ckpt
epoch 4, training accuracy 0.859375
test accuracy 0.7133333086967468
model saved to ./models/4.ckpt
epoch 5, training accuracy 0.90625
test accuracy 0.7666666507720947
model saved to ./models/5.ckpt
epoch 6, training accuracy 0.875
test accuracy 0.7766666412353516
model saved to ./models/6.ckpt
epoch 7, training accuracy 0.9375
test accuracy 0.7766666412353516
model saved to ./models/7.ckpt
epoch 8, training accuracy 0.84375
test accuracy 0.846666693687439
model saved to ./models/8.ckpt
epoch 9, training accuracy 0.859375
test accuracy 0.8266666531562805
model saved to ./models/9.ckpt
epoch 10, training accuracy 0

In [ ]:
int(10.5/2)

In [ ]:
# with tf.Session() as sess:
#     saver = tf.train.import_meta_graph('./models/30.ckpt.meta')
#     saver.restore(sess, './models/30.ckpt')
#     accuracy = tf.get_collection('accuracy')[0]
#     x_in = tf.get_collection('x_in')[0]
#     y_in = tf.get_collection('y_in')[0]
#     print('model restored')
#     print('test accuracy {}'.format(accuracy.eval(session=sess, feed_dict={x_in:dev_x[0:64], y_in:dev_y[0:64]})))

In [ ]:
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     writer = tf.summary.FileWriter('../logs',sess.graph)
#     accuracy_res = accuracy.eval(session=sess, feed_dict={x_in:train_x[0:10], y_in:train_y[0:10]})
#     print(accuracy_res)
#     writer.close()